In [2]:
import numpy as np
import pandas as pd
import re
import time
from tqdm.notebook import tqdm

# 거리계산
from haversine import haversine #패키지 설치필요

In [3]:
### 파일 불러오기
file_path = './data/'

sleep = pd.read_excel(file_path+'졸음쉼터+편의시설.xlsx')
rest = pd.read_csv(file_path+'휴게소정보_20210809.csv', encoding='euc-kr')
rest = rest.iloc[:, :-4]
for val in rest.columns[12:]:
    rest = rest.rename(columns={val:val[:-2]})
danger = pd.read_csv(file_path+'danger.csv')

In [4]:
df_danger4 = danger[danger['danger_rank']==4]
col = ['위도', '경도', '졸음쉼터명']
sleep_crush_bound = pd.DataFrame([np.zeros(len(col))], columns=col)
cnt = 0

### 졸음쉼터 500m이내 사고가 발생한 좌표 모두 저장하는 df만들기
for j in range(len(df_danger4)):
    b = [df_danger4.iloc[j, -2], df_danger4.iloc[j, -1]]
    
    for i in range(len(sleep)):
        aaa = [sleep['위도'][i], sleep['경도'][i]]
        if 500 > haversine(b, aaa, unit = 'm'):
            sleep_crush_bound.loc[cnt] = [sleep['위도'][i], sleep['경도'][i], sleep.iloc[i,0]]
            cnt +=1
sleep_crush_bound

,위도,경도,졸음쉼터명
0,36.156517,128.02649,봉산
1,36.156432,128.02568,봉산
2,36.156432,128.02568,봉산
3,36.156517,128.02649,봉산
4,36.156517,128.02649,봉산
...,...,...,...
251,36.156517,128.02649,봉산
252,36.156517,128.02649,봉산
253,36.156432,128.02568,봉산
254,36.156432,128.02568,봉산


In [5]:
dic = dict(sleep_crush_bound['졸음쉼터명'].value_counts())
sleep_crush_bound_cnt = sleep_crush_bound.groupby('졸음쉼터명').head(1)
sleep_crush_bound_cnt['cnt'] = dic.values()
sleep_crush_bound_cnt = sleep_crush_bound_cnt.reset_index(drop=True)

### 같은 졸음쉼터명끼리 count
for i in range(len(sleep_crush_bound_cnt)):
    sleep_crush_bound_cnt['cnt'][i] = dic[sleep_crush_bound_cnt['졸음쉼터명'][i]]

sleep_crush_bound_cnt
# sleep_crush_bound_cnt.to_excel('./data/졸음쉼터_주변사고.xlsx')

C:\Users\USER\AppData\Local\Temp\ipykernel_13224\2813439173.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_crush_bound_cnt['cnt'] = dic.values()
C:\Users\USER\AppData\Local\Temp\ipykernel_13224\2813439173.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_crush_bound_cnt['cnt'][i] = dic[sleep_crush_bound_cnt['졸음쉼터명'][i]]


,위도,경도,졸음쉼터명,cnt
0,36.156517,128.026490,봉산,64
1,36.051633,128.408056,석적,8
2,37.096192,127.120738,남사,16
3,36.152819,128.190756,대신,32
4,37.156708,127.087165,오산,64
5,35.545332,129.118872,삼남,14
6,36.914432,127.188465,입장,8
7,37.001112,127.151971,원곡,10
8,35.342356,129.042148,양산,24
9,36.655882,127.372294,옥산,16
